In [68]:
import boto3
import json

## Create IAM role for Lambda function and attach Sagemaker priviliges to it

In [69]:
iam_client = boto3.client('iam')

In [70]:
role_policy = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "",
      "Effect": "Allow",
      "Principal": {
        "Service": "lambda.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}

In [71]:
iam_rolename = "LambdaHFServerlessRole"

In [72]:
response = iam_client.create_role(
  RoleName=iam_rolename,
  AssumeRolePolicyDocument=json.dumps(role_policy),
)

In [73]:
response = iam_client.attach_role_policy(RoleName=iam_rolename, PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess')

## Create Lambda function

In [74]:
endpoint_name = "hf-serverless-ep2021-12-23-10-42-53"

lambda_code = f"""
import boto3
import json

ENDPOINT_NAME = "{endpoint_name}"
runtime= boto3.client('runtime.sagemaker')

def lambda_handler(event, context):
    print("Received event: " + json.dumps(event['body'], indent=4))
    
    payload = {{"inputs": event['body']}}
    response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                       ContentType='application/json',
                                       Body=json.dumps(payload))
    result = json.loads(response['Body'].read().decode())
    pred = result[0]['label']
    score = result[0]['score']
    
    if pred == "LABEL_0":
        final_result = f"Negative sentiment, Probability: {{round(score * 100, 1)}}%"
    else:
        final_result = f"Positive sentiment, Probability: {{round(score * 100, 1)}}%"

    return final_result
"""

In [75]:
filename = "lambda"

In [76]:
with open(f"{filename}.py", "w") as f:
    f.write(lambda_code)

In [77]:
from zipfile import ZipFile

In [78]:
with ZipFile('lambda.zip', 'w') as myzip:
    myzip.write('lambda.py')

In [79]:
lambda_function_name = "HFServerlessLambdaFunction"

In [82]:
lambda_client = boto3.client('lambda')

with open('lambda.zip', 'rb') as f:
    zipped_code = f.read()
  
role = iam_client.get_role(RoleName=iam_rolename)

response = lambda_client.create_function(
    FunctionName=lambda_function_name,
    Runtime='python3.9',
    Role=role['Role']['Arn'],
    Handler=f'{filename}.lambda_handler',
    Code=dict(ZipFile=zipped_code),
    Timeout=300,
)

In [83]:
lambda_arn = response['FunctionArn']
lambda_arn

'arn:aws:lambda:us-east-1:905847418383:function:HFServerlessLambdaFunction'

## Create HTTP API

In [87]:
api_client = boto3.client('apigatewayv2')

In [88]:
api_name = "HFServerlessAPI"

In [89]:
response = api_client.create_api(
    Name=api_name,
    ProtocolType='HTTP',
    Target=lambda_arn,
    RouteKey='ANY /test',
)

In [90]:
api_id = response['ApiId']
api_id

'93llsq9a65'

In [91]:
response = lambda_client.add_permission(
    FunctionName=lambda_function_name,
    StatementId='HFServerlessStatementID',
    Action='lambda:InvokeFunction',
    Principal='apigateway.amazonaws.com',
    SourceArn=f'arn:aws:execute-api:us-east-1:905847418383:{api_id}/*/*/test',
)

### Test API

In [92]:
import requests

In [93]:
url = f'https://{api_id}.execute-api.us-east-1.amazonaws.com/test'
data = 'This is a great test'

response = requests.post(url, json = data)
response.text

'Positive sentiment, Probability: 98.6%'

## Cleanup

In [ ]:
# iam_client.detach_role_policy(RoleName=iam_rolename, PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess')
# iam_client.delete_role(RoleName=iam_rolename)
# lambda_client.delete_function(FunctionName=lambda_function_name)
# api_client.delete_api(ApiId=api_id)